In [1]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=16)
cluster

VBox(children=(HTML(value='<h2>KubeCluster</h2>'), HBox(children=(HTML(value='\n<div>\n  <style scoped>\n    .dataframe tbody tr th:only-of-type {\n        vertical-align: middle;\n    }\n\n    .dataframe tbody tr th {\n        vertical-align: top;\n    }\n\n    .dataframe thead th {\n        text-align: right;\n    }\n  </style>\n  <table style="text-align: right;">\n    <tr><th>Workers</th> <td>0</td></tr>\n    <tr><th>Cores</th> <td>0</td></tr>\n    <tr><th>Memory</th> <td>0 B</td></tr>\n  </table>\n</div>\n', layout=Layout(min_width='150px')), Accordion(children=(HBox(children=(IntText(value=0, description='Workers', layout=Layout(width='150px')), Button(description='Scale', layout=Layout(width='150px'), style=ButtonStyle()))), HBox(children=(IntText(value=0, description='Minimum', layout=Layout(width='150px')), IntText(value=0, description='Maximum', layout=Layout(width='150px')), Button(description='Adapt', layout=Layout(width='150px'), style=ButtonStyle())))), layout=Layout(min_width='500px'), selected_index=None, _titles={'0': 'Manual Scaling', '1': 'Adaptive Scaling'}))), HTML(value='<p><b>Dashboard: </b><a href="/user/andersy005/proxy/8787/status" target="_blank">/user/andersy005/proxy/8787/status</a></p>\n')))

In [3]:
from dask.distributed import Client
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://10.20.168.12:46623 Dashboard: /user/andersy005/proxy/8787/status,Cluster Workers: 14 Cores: 28 Memory: 84.00 GB


In [5]:
import dask.dataframe as dd

In [6]:
dtypes = {
        'ip':'category',
        'app': 'category',
        'device': 'category',
        'os': 'category',
        'channel': 'category',
        'is_attributed': 'uint8'
        }

In [7]:
df = dd.read_csv('gcs://kaggle-talkingdata-bucket/train.csv', 
                 parse_dates=['click_time', 'attributed_time'], 
                 dtype=dtypes)

In [8]:
df.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaT,0
1,17357,3,1,19,379,2017-11-06 14:33:34,NaT,0
2,35810,3,1,13,379,2017-11-06 14:34:12,NaT,0
3,45745,14,1,13,478,2017-11-06 14:34:52,NaT,0
4,161007,3,1,13,379,2017-11-06 14:35:08,NaT,0


In [9]:
from collections import Counter

In [10]:
Counter(df.is_attributed.compute())

Counter({0: 184447044, 1: 456846})

distributed.core - WARNING - Event loop was unresponsive in Scheduler for 11.83s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.


In [11]:
df1 = df[df.is_attributed == 0]
df1.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaT,0
1,17357,3,1,19,379,2017-11-06 14:33:34,NaT,0
2,35810,3,1,13,379,2017-11-06 14:34:12,NaT,0
3,45745,14,1,13,478,2017-11-06 14:34:52,NaT,0
4,161007,3,1,13,379,2017-11-06 14:35:08,NaT,0


In [12]:
len(df1)

184447044

In [13]:
df2 = df[df.is_attributed == 1]
df2.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
103,204158,35,1,13,21,2017-11-06 15:41:07,2017-11-07 08:17:19,1
1504,29692,9,1,22,215,2017-11-06 16:00:02,2017-11-07 10:05:22,1
1798,64516,35,1,13,21,2017-11-06 16:00:02,2017-11-06 23:40:50,1
2102,172429,35,1,46,274,2017-11-06 16:00:03,2017-11-07 00:55:29,1
3056,199085,35,1,13,274,2017-11-06 16:00:04,2017-11-06 23:04:54,1


In [14]:
len(df2)

456846

In [15]:
df3 = df1.sample(0.003, random_state=42)
len(df3)

553338

In [16]:
df3.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
1026097,85171,18,1,22,121,2017-11-06 16:22:27,NaT,0
801951,7595,15,1,32,245,2017-11-06 16:17:08,NaT,0
1147434,15680,27,1,9,153,2017-11-06 16:25:33,NaT,0
307737,19902,3,1,13,280,2017-11-06 16:05:54,NaT,0
76037,125141,6,1,19,459,2017-11-06 16:01:21,NaT,0


In [17]:
data = dd.concat([df2, df3], axis=0)
data.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
103,204158,35,1,13,21,2017-11-06 15:41:07,2017-11-07 08:17:19,1
1504,29692,9,1,22,215,2017-11-06 16:00:02,2017-11-07 10:05:22,1
1798,64516,35,1,13,21,2017-11-06 16:00:02,2017-11-06 23:40:50,1
2102,172429,35,1,46,274,2017-11-06 16:00:03,2017-11-07 00:55:29,1
3056,199085,35,1,13,274,2017-11-06 16:00:04,2017-11-06 23:04:54,1


In [18]:
len(data)

1010184

In [19]:
data = client.persist(data.repartition(npartitions=1))

In [20]:
df = data.compute()

In [21]:
df.to_parquet('data/talking_data.parquet')

In [22]:
client.close()

In [23]:
client

<Client: not connected>